<a href="https://colab.research.google.com/github/sh1var3ddy/CNN/blob/main/Fastrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [26]:
! kaggle datasets download -d saife245/tuberculosis-image-datasets

tuberculosis-image-datasets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [27]:
!unzip /content/tuberculosis-image-datasets.zip

Archive:  /content/tuberculosis-image-datasets.zip
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0001.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0001.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0002.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0002.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0003.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0003.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0004.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0004.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0005.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0005.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0006.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0006.xml  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0007.jpg  
  inflating: tuberculosis-phonecamera/tuberculosis-phone-0007.xml

In [28]:

import shutil
import os

In [29]:
!cd /content/tuberculosis-phonecamera/ && mkdir images && mkdir annot

mkdir: cannot create directory ‘images’: File exists


In [30]:
!mv /content/tuberculosis-phonecamera/*.jpg  /content/tuberculosis-phonecamera/images

In [31]:

!mv /content/tuberculosis-phonecamera/*.xml  /content/tuberculosis-phonecamera/annot

In [32]:

!mkdir train && mkdir test

mkdir: cannot create directory ‘train’: File exists


In [33]:
!mkdir val

mkdir: cannot create directory ‘val’: File exists


In [34]:
!cd /content/train/ && mkdir images && mkdir annot
!mkdir train/annotjson

mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘train/annotjson’: File exists


In [35]:
!cd /content/test/ && mkdir images && mkdir annot
!cd /content/val/ && mkdir images && mkdir annot

mkdir: cannot create directory ‘images’: File exists
mkdir: cannot create directory ‘images’: File exists


In [36]:
# for i in range(350):
#   shutil.move(f"/content/tuberculosis-phonecamera/images/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.jpg","/content/train/images/")
#   shutil.move(f"/content/tuberculosis-phonecamera/annot/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.xml","/content/train/annot/")
# for i in range(350,425):
#   shutil.move(f"/content/tuberculosis-phonecamera/images/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.jpg","/content/test/images/")
#   shutil.move(f"/content/tuberculosis-phonecamera/annot/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.xml","/content/test/annot/")
# for i in range(425,500):
#   shutil.move(f"/content/tuberculosis-phonecamera/images/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.jpg","/content/val/images/")
#   shutil.move(f"/content/tuberculosis-phonecamera/annot/tuberculosis-phone-{str('0'*(4-len(str(i+1)))+str(i+1))}.xml","/content/val/annot/")

In [37]:
# pip install xmltodict

In [38]:
import json
import xmltodict
for f in os.listdir("/content/train/annot"):
  with open("/content/train/annot/"+f) as xml_file:
      
      data_dict = xmltodict.parse(xml_file.read())
      xml_file.close()
      
      json_data = json.dumps(data_dict)
      with open("/content/train/annotjson."+"json", "a") as json_file:
          json_file.write(json_data)
          json_file.close()

In [39]:
# !which pip3

In [40]:
# pip install detectron2 -f \https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html


In [41]:
import detectron2

In [42]:
import torch, torchvision

print(torch.__version__, torch.cuda.is_available())
!gcc --version


1.11.0+cu113 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [43]:
torch.cuda.is_available()

True

In [44]:
# !pip3 install histomicstk --find-links https://girder.github.io/large_image_wheels

In [45]:
# #normalization
import matplotlib.pyplot as plt
import histomicstk as htk
#Scikit-image, or skimage, is an open source Python package designed for image preprocessing.
import skimage.io
import skimage.measure
import skimage.color
ref_image_file = ('/content/train/images/tuberculosis-phone-0001.jpg')
im_reference = skimage.io.imread(ref_image_file)[:, :, :3]
# fig,a=plt.subplots(1,2,figsize=(40,40))
print(im_reference.shape)

def normalize(im_input):
    mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(im_reference)
    im_nmzd = htk.preprocessing.color_normalization.reinhard(im_input, mean_ref, std_ref)
    return im_nmzd

#######    Normalising all the images according to histomicstk  ########


(1224, 1632, 3)


In [56]:
import cv2
import glob
from tqdm import tqdm

In [57]:
test=glob.glob("/content/test/images/*.jpg")
train=glob.glob("/content/train/images/*.jpg")
valid=glob.glob("/content/val/images/*.jpg")

In [58]:
print(len(test),len(train),len(valid))

75 350 75


In [59]:
# for img in tqdm(train):
#     cv2.imwrite(img,normalize(cv2.imread(img)))

100%|██████████| 350/350 [04:44<00:00,  1.23it/s]


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import matplotlib.pyplot as plt
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances
